In [33]:
import numpy as np
import numpy.linalg as la

# Cholesky-Zerlegung
## (a)

#### Definition der Matrix A

In [34]:
A = np.array([
    [4,2,-1,0,0,0],
    [2,4,1,1,0,1],
    [-1,1,5,3,-1,2],
    [0,1,3,4,1.1,2.5],
    [0,0,-1,1.1,2.4,1],
    [0,1,2,2.5,1,3]])
print(A)

[[ 4.   2.  -1.   0.   0.   0. ]
 [ 2.   4.   1.   1.   0.   1. ]
 [-1.   1.   5.   3.  -1.   2. ]
 [ 0.   1.   3.   4.   1.1  2.5]
 [ 0.   0.  -1.   1.1  2.4  1. ]
 [ 0.   1.   2.   2.5  1.   3. ]]


Kriterium für positiv definit: all Eigenwerte > 0

In [35]:
D,V = la.eig(A)  #Eigenwerte, Eigenvektoren
print(D)
print("Ist A positiv definit?: " + str((D > 0).all()))

[ 9.70612261  5.99083183  3.86413908  1.64650095  0.25881123  0.9335943 ]
Ist A positiv definit?: True


## (b)

#### Funktionsdefinition

In [36]:
def CholeskyZerlegung(A):
    """
    Berechnet die Cholesky-Zerlegung der (symmetrisch & pos. def.) Matrix A 
    Rückgabewert is the untere Dreiecksmatrix L
    """
    n = A.shape[0]
    L = np.zeros_like(A)
        
    for j in range(n):
        # Schritt 1: Diagonalelement berechnen
        i = j
        L[i][j] = np.sqrt(A[i][i] - sum([L[i][k]**2 for k in range(i)]))
        # Schritt 2: Spalte von L nach unten auffüllen
        for i in range(j+1,n):
            L[i][j] = (A[i][j] - sum(L[i][k]*L[j][k] for k in range(j)))/L[j][j]
    
    return L

#### Berechnung L

In [37]:
L = CholeskyZerlegung(A)
print(L)

[[ 2.          0.          0.          0.          0.          0.        ]
 [ 1.          1.73205081  0.          0.          0.          0.        ]
 [-0.5         0.8660254   2.          0.          0.          0.        ]
 [ 0.          0.57735027  1.25        1.4505746   0.          0.        ]
 [ 0.          0.         -0.5         1.18918393  0.85781209  0.        ]
 [ 0.          0.57735027  0.75        0.84736536  0.42821352  1.09670953]]


#### Überprüfen

In [38]:
R = A-L.dot(L.T)
print("Residium nach Zerlegung:")
print("||R||f = {}".format(la.norm(R,ord='fro')))

Residium nach Zerlegung:
||R||f = 4.440892098500626e-16


## (c)

#### Funktionsdefinition

In [39]:
def CholeskyLösung(L,y):
    """
    Berechnet die Lösung des LGS Ax=y aus der Cholesky Zerlegung der Matrix A und der rechten Seite y.
    """
    n = A.shape[0]
    x=np.zeros(n)
    x.shape = (n,1)  # x explizit zu Spaltenvektor
    
    z =np.array(x)
    
    #vorwärtseinsetzen L.dot(z) = y
    z[0] = y[0]/L[0][0]
    for i in range(1,n):
        z[i] = (y[i]-sum([L[i][k]*z[k] for k in range(i)]))/L[i][i]
    
    #rückwärtseinsetzen L.T.dot(x) = z
    x[n-1] = z[n-1]/L[n-1][n-1]
    for i in reversed(range(0,n-1)):
        x[i] = (z[i]-sum([L[k][i]*x[k] for k in range(i+1,n)]))/L[i][i]
    return x

#### Lösung x

In [40]:
y = np.array([[1],[-1],[5],[7],[6],[-3]])
x = CholeskyLösung(L,y)
print(x)

[[ 2.90010068]
 [-1.32923146]
 [ 7.94193981]
 [-1.81787672]
 [ 9.81205951]
 [-7.60733863]]


#### Residuum

In [41]:
r = A.dot(x)-y
print(r)

[[  0.00000000e+00]
 [  0.00000000e+00]
 [ -3.55271368e-15]
 [  0.00000000e+00]
 [  1.77635684e-15]
 [ -7.10542736e-15]]


In [42]:
print("||r||1 = {}".format(la.norm(r,ord=1)))
print("||r||2 = {}".format(la.norm(r,ord=2)))
print("||r||inf = {}".format(la.norm(r,ord=np.inf)))

||r||1 = 1.2434497875801753e-14
||r||2 = 8.140289677804162e-15
||r||inf = 7.105427357601002e-15
